In [0]:
df = (spark.readStream
  .format("cloudFiles")
  .option("cloudFiles.format", "json")
  .load("abfss://container@storageaccount.dfs.core.windows.net/input-path/"))

(df.writeStream
  .format("delta")
  .option("checkpointLocation", "abfss://container@storageaccount.dfs.core.windows.net/checkpoints/")
  .start("abfss://container@storageaccount.dfs.core.windows.net/output-table/"))


In [0]:
# Input path for JSON files (e.g., raw landing zone)
input_path = "abfss://container@storageaccount.dfs.core.windows.net/raw/json/"

# Schema evolution and checkpoint paths
schema_location = "abfss://container@storageaccount.dfs.core.windows.net/schema/"
checkpoint_location = "abfss://container@storageaccount.dfs.core.windows.net/checkpoints/"

# Streaming DataFrame with Auto Loader
raw_df = (spark.readStream
  .format("cloudFiles")
  .option("cloudFiles.format", "json")
  .option("cloudFiles.schemaLocation", schema_location)
  .option("cloudFiles.schemaEvolutionMode", "rescue")
  .load(input_path)
)

# Optional: Apply transformations (e.g., parse nested JSON)
# transformed_df = raw_df.select(...)

# Write to Delta table
query = (raw_df.writeStream
  .format("delta")
  .outputMode("append")
  .option("checkpointLocation", checkpoint_location)
  .trigger(availableNow=True)  # For batch-like processing; use processingTime for continuous
  .start("abfss://container@storageaccount.dfs.core.windows.net/delta/customers/")
)

query.awaitTermination()
